In [38]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fftshift, ifftn
from mpl_toolkits.mplot3d import Axes3D

In [35]:
def UShear(z, z_ref=100, u_inf=10, model="powerlaw", z0=0.03):
    if model == "powerlaw":
        return (z / z_ref) ** (0.14285714285714285) * u_inf
    elif model == "loglaw":
        kappa = 0.4  # von Karman constant
        U_star = u_inf / (np.log(z_ref / z0) / kappa)  # Friction velocity
        return (U_star / kappa) * np.log(z / z0)
    
def VonKarmanTurbulenceField(Lx, Ly, Lz, Nx, Ny, Nz, L):
    # mesh for the wave number vector k
    ky, kz, kx = np.meshgrid((np.pi / Ly) * np.arange(-Ny, Ny, 2), 
                             (np.pi / Lz) * np.arange(-Nz, Nz, 2), 
                             (np.pi / Lx) * np.arange(-Nx, Nx, 2), 
                             indexing='ij')
    
    # power spectrum, the exponent already contains the factor 1/2 for the square root
    S = (L**(-2) + kx**2 + ky**2 + kz**2)**(-17/12)
    
    # random phase spectra with uniformly distributed phases in [0, 2pi]
    PhiX = np.exp(2j * np.pi * np.random.rand(Ny, Nz, Nx))
    PhiY = np.exp(2j * np.pi * np.random.rand(Ny, Nz, Nx))
    PhiZ = np.exp(2j * np.pi * np.random.rand(Ny, Nz, Nx))
    
    # inverse Fourier transform, using the cross product with the k vector
    u = ifftn(fftshift(S * (ky * PhiZ - kz * PhiY)), s=(Ny, Nz, Nx)).real
    v = ifftn(fftshift(S * (kz * PhiX - kx * PhiZ)), s=(Ny, Nz, Nx)).real
    w = ifftn(fftshift(S * (kx * PhiY - ky * PhiX)), s=(Ny, Nz, Nx)).real
    
    return u, v, w

def integrate_over_disk(f, R, nodes, weights):
    result = 0
    for i in range(len(nodes)):
        r = R * nodes[i][2]  # scaling the radius
        phi = nodes[i][3]     # angle
        w = R**2 * weights[i] # scaled weight
        result += w * f(r, phi)
    return result



def solve_dynamics_in_fourier(M, gamma, K, f_t, dt):
    # Fourier transform of forcing
    F_omega = np.fft.fft(f_t)
    
    # Frequency array
    omega = np.fft.fftfreq(len(f_t), dt) * 2 * np.pi
    
    # Solving in Fourier space
    epsilon = 1e-8  # Small damping factor to avoid division by zero
    X_omega = F_omega / (-omega**2 * M + 1j * omega * gamma + K + epsilon)

    
    # Inverse Fourier transform to get back to time domain
    x_t = np.fft.ifft(X_omega).real
    
    return x_t


In [47]:

# Parameters based on your assignment
U = 10  # Transport velocity in m/s
Lx = 6000  # Length of the domain in the x-direction (600s * 10ms/s = 6000 m)
Ly = 384   # Width in the y-direction (384 m)
Lz = 384   # Height in the z-direction (384 m)
Nx = 8192  # Number of grid points in x-direction
Ny = 64    # Number of grid points in y-direction
Nz = 64    # Number of grid points in z-direction
L = 35.4   # Integral length scale in meters

# Generate the turbulent velocity fields
u_turb, v_turb, w_turb = VonKarmanTurbulenceField(Lx, Ly, Lz, Nx, Ny, Nz, L)